In [34]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import csv
import json
import pandas as pd

In [2]:
root_url = 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/'

In [3]:
r = requests.get(root_url)

In [4]:
soup = BeautifulSoup(r.content, 'html.parser')

# Get all links

In [24]:
def get_all_links(root_url):
    '''
    Funzione che prende tutte le pagine a partire dal link della pagina 1 del forum o del thread
    '''

    links = [root_url]
    r = requests.get(root_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    list_pag = soup.select('ul.pageNav-main li')
    if list_pag:
        pag_max = soup.select('ul.pageNav-main li')[-1].a.text
        pag_max = int(pag_max)
        for page in range(2,pag_max+1):
            nurl = f'{root_url}page-{page}'
            links.append(nurl)
        
    return links

In [25]:
lista_pagine = get_all_links('https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/')

In [26]:
lista_pagine

['https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-2',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-3',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-4',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-5',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-6',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-7',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-8',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-9',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-10',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-11',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-12',
 'https://ilforumdeibrutti.is/index.php?forums/una-vita-da-brutto.6/page-13',
 

In [27]:
def get_all_threads(root_url,lista_pags):

    '''
    Prende in input un root_url VERO. Ad esempio https://ilforumdeibrutti.is 
    e una lista di pagine di elenchi di threads. Per ogni pagina estrae il link della pagina 1 di ogni thread
    '''

    #attenzione! root_url in questo caso è diverso da quello precedente
    #per forum brutti è https://ilforumdeibrutti.is
    lista_totale_threads = []
    for pagina in lista_pags:
        r = requests.get(pagina)
        soup = BeautifulSoup(r.content, 'html.parser')
        lista_threads_pagina = list(map(lambda x: root_url+x['href'],soup.select('.structItem-title a')))
        lista_totale_threads.extend(lista_threads_pagina)
    
    return lista_totale_threads
        

In [28]:
all_threads = get_all_threads('https://ilforumdeibrutti.is',lista_pagine)

In [29]:
all_pages_all_threads = []
for thread in tqdm(all_threads):
    pagine_threads = get_all_links(thread) #per ogni thread, estraiamo i link di TUTTE LE PAGINE DEL THREAD
    all_pages_all_threads.extend(pagine_threads)

  0%|          | 0/2400 [00:00<?, ?it/s]

100%|██████████| 2400/2400 [11:13<00:00,  3.56it/s]


In [30]:
len(all_threads)

2400

In [40]:
len(all_pages_all_threads)

4403

In [37]:
import json

In [38]:
with open('lista_threads_fdb.json','w') as file: #salviamo perché ci mette tanto
    json.dump(all_pages_all_threads, file)

# Riprendiamo la lista

In [39]:
with open('lista_threads_fdb.json','r') as file:
    all_pages_all_threads = json.load(file)

In [42]:
def scrape_posts(pages_threads, file_path):

    '''
    Prende in input una lista con tutte le pagine di tutti i thread. 
    Per ogni pagina estrae i post e salva tutto in un file
    '''

    for thread in tqdm(pages_threads):
        r = requests.get(thread)
        soup = BeautifulSoup(r.content,'html.parser')
        titolo = soup.select_one('.p-title-value').text
        postss = soup.select('article.message')
        for post in postss:
            autore = post['data-author']
            data = post.select_one('time')['datetime']
            elem_testo = post.select_one('.bbWrapper')
            testo = ''.join(elem_testo.find_all(string=True,recursive=False))

            with open(file_path,'a',newline="", encoding="UTF-8") as file:
                csv_writer = csv.writer(file, delimiter=',', quoting=csv.QUOTE_ALL)
                riga = [titolo, autore, data, testo]
                csv_writer.writerow(riga)

In [43]:
scrape_posts(all_pages_all_threads,'./fdb_scraped_pulito.csv') #ecco qua, ci ha messo 29 minuti!!

100%|██████████| 4403/4403 [24:33<00:00,  2.99it/s]  
